# Clustering Assignment

## Requirements

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto. Start by creating a new Notebook for this assignment. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [3]:
# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
print('Libraries imported.')

Libraries imported.


In [4]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'lxml')
#print(soup.prettify())

In [5]:
# extracting the raw table inside that webpage
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Step 1B: Data Cleanup
<ol>
    <li>Remove Boroughs that are 'Not assigned'</li>
<li>More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma</li>
<li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough</li>
    </ol>

In [6]:
#Remove Boroughs that are 'Not assigned'
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
# More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
canada_df["Neighborhood"] = canada_df.groupby("Postal Code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))

#remove duplicates
canada_df = canada_df.drop_duplicates()

#update index to be postcode if it isn't already
if(canada_df.index.name != 'Postal Code'):
    canada_df = canada_df.set_index('Postal Code')
    
canada_df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
canada_df['Neighborhood'].replace("Not assigned", canada_df["Borough"],inplace=True)
canada_df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
canada_df.shape

(103, 2)

# Question 2

## Use the Geocoder package or the csv file to create dataframe with longitude and latitude values

We will be using a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [10]:
geo_url="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(geo_url)

In [11]:
#geo_data.columns
geo_data.columns=['Postal Code', 'Latitude', 'Longitude']

In [12]:
toronto_df2= pd.merge(canada_df, geo_data, how='inner', on="Postal Code")
toronto_df2

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [13]:
toronto_df2.shape

(103, 5)

# Question 3
## Explore and cluster the neighborhoods in Toronto

In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df2['Borough'].unique()),
        toronto_df2.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [28]:
from geopy.geocoders import Nominatim 
import geopy
# convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [30]:
#get Latitute and longitude of toronto

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ON")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto City are 43.6534817, -79.3839347.


In [31]:
# create map of Torronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df2['Latitude'], toronto_df2['Longitude'], toronto_df2['Borough'], toronto_df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto

In [32]:
df_borough_toronto=toronto_df2[toronto_df2["Borough"].str.contains("Toronto")].reset_index(drop=True)
df_borough_toronto.size

195

In [33]:
df_borough_toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [34]:
df_borough_toronto["Borough"].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

In [35]:
df_borough_toronto["color"]=df_borough_toronto["Borough"].map({'East Toronto':"green", 'Central Toronto':"red", 'Downtown Toronto':"blue",
       'West Toronto':"black"})

In [36]:
df_borough_toronto.shape

(39, 6)

In [37]:

CLIENT_ID = '5TFV3HSGSS0XSH4MEWPBEBSVBVVX5PWSSQFME1KIUZHEVLSF' # your Foursquare ID
CLIENT_SECRET = 'OA0N3H5CPBENZ5WIOZDOKWVMODM25LY2QIIVHA3AIHEHBHCP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5TFV3HSGSS0XSH4MEWPBEBSVBVVX5PWSSQFME1KIUZHEVLSF
CLIENT_SECRET:OA0N3H5CPBENZ5WIOZDOKWVMODM25LY2QIIVHA3AIHEHBHCP


In [38]:
#first neigbourhood
neighborhood_latitude1=df_borough_toronto["Latitude"][0]
neighborhood_longitude1=df_borough_toronto["Longitude"][0]
neighborhood_name1=df_borough_toronto["Neighborhood"][0]

print (f"{neighborhood_name1} has lognitude and latitude as : [{neighborhood_latitude1},{neighborhood_longitude1}]")

Regent Park, Harbourfront has lognitude and latitude as : [43.6542599,-79.3606359]


In [39]:
# Setup API URL to explore venues near by
LIMIT=100
RADIUS=500
url=f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={neighborhood_latitude1},{neighborhood_longitude1}&v={VERSION}&radius={RADIUS}&limit={LIMIT}"
neighborhood_json = requests.get(url).json()["response"]["groups"][0]["items"]

In [40]:

# Serializing json
import json
json_object = json.dumps(neighborhood_json, indent = 4)

In [41]:
#save data as json file to explore
with open("jsonData.json","w") as f:
    f.write(json_object)

In [43]:
venues=neighborhood_json

In [46]:
#flatten Json
from pandas import json_normalize
nearby_venues=json_normalize(venues)

In [47]:
filtered_columns=['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

In [48]:
nearby_venues=nearby_venues.loc[:,filtered_columns]

In [49]:
def getCategory_type(row):
    try:
        category_list=row["name"]
    except:
        category_list=row["venue.categories"]
    if len(category_list)==0:
        return None
    else:
        return category_list[0]["name"]

In [50]:
nearby_venues["categories"]= [x[0]["name"] for x in nearby_venues["venue.categories"]]

In [51]:
nearby_venues.drop(["venue.categories"],axis=1,inplace=True)

In [52]:
nearby_venues

,venue.name,venue.location.lat,venue.location.lng,categories
0,Roselle Desserts,43.653447,-79.362017,Bakery
1,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Impact Kitchen,43.656369,-79.356980,Restaurant
5,Corktown Common,43.655618,-79.356211,Park
6,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
7,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
8,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
9,The Distillery Historic District,43.650244,-79.359323,Historic Site


In [53]:
df_borough_toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude,color
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,blue
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,blue
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,blue
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,blue
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,green
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,blue
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,blue
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,blue
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,blue
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,black


In [55]:
def getNearByVenues(neighbourhood_name,lat,long):
    venues_list=[]

    for name, lat, lng in zip(neighbourhood_name,lat,long):
        print(name)
        
        url=f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={lat},{lng}&v={VERSION}&radius={RADIUS}&limit={LIMIT}"
        neighborhood_json = requests.get(url).json()["response"]["groups"][0]["items"]
        venues_list.append([(
            name,
            lat,
            lng,
            v["venue"]["name"],
            v["venue"]["location"]["lat"],
            v["venue"]["location"]["lng"],
            v["venue"]["categories"][0]["name"]) for v in neighborhood_json])
        #appending list of  venuedetails as list into another list venues list
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Neighborhood', 
                    'Neighborhood Latitude', 
                    'Neighborhood Longitude', 
                    'Venue', 
                    'Venue Latitude', 
                    'Venue Longitude', 
                    'Venue Category']
    return (nearby_venues)

In [56]:
toronto_venues_df = getNearByVenues(df_borough_toronto['Neighborhood'],df_borough_toronto['Latitude'],df_borough_toronto['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [59]:
toronto_venues_df.shape

(1620, 7)

In [60]:
toronto_venues_df.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,68,68,68,68,68,68
Christie,17,17,17,17,17,17
Church and Wellesley,70,70,70,70,70,70
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37


In [61]:
#analyze the neighbourhoood
#creating dummy for each venue category

torento_onehot=pd.get_dummies(toronto_venues_df[["Venue Category"]], prefix="", prefix_sep="")

In [62]:
torento_onehot.shape

(1620, 229)

In [63]:
torento_onehot["Neighborhood"]=toronto_venues_df["Neighborhood"]

In [64]:
torento_onehot.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
torento_onehot.columns.get_loc("Neighborhood")

158

In [66]:

torento_onehot.columns[158]

'Neighborhood'

In [67]:
fixed_columns=[torento_onehot.columns[159]]+list(torento_onehot.columns[0:159])+list(torento_onehot.columns[160:])

In [68]:
len(fixed_columns)

229

In [69]:
torento_onehot=torento_onehot[fixed_columns]

In [70]:

torento_onehot.head()

,New American Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
torento_onehot_grouped=torento_onehot.groupby("Neighborhood").mean().reset_index()

In [72]:
torento_onehot_grouped

,Neighborhood,New American Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.014706,0.000000,0.000000,0.014706,0.000000,0.014706
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,0.000000,...,0.014286,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571
7,"Commerce Court, Victoria Hotel",0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.027027,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [74]:

for  hood in torento_onehot_grouped["Neighborhood"]:
    print(f"-------{hood}----")
    temp=torento_onehot_grouped[torento_onehot_grouped["Neighborhood"]==hood].T.reset_index()
    temp.columns=["venue","freq"]
    temp=temp[1:]
    temp["freq"]=round(temp["freq"].astype(float),2)
    print(temp.sort_values(by="freq",axis=0,ascending=False).reset_index(drop=True).head(10))
    dict1={}
    print("\n")

-------Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2                Café  0.04
3         Cheese Shop  0.04
4  Seafood Restaurant  0.04
5            Beer Bar  0.04
6          Restaurant  0.04
7              Bakery  0.04
8      Farmers Market  0.04
9    Greek Restaurant  0.02


-------Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.13
1                  Bakery  0.09
2          Breakfast Spot  0.09
3             Coffee Shop  0.09
4  Furniture / Home Store  0.04
5                     Bar  0.04
6                 Stadium  0.04
7            Climbing Gym  0.04
8              Restaurant  0.04
9      Italian Restaurant  0.04


-------Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0  Gym / Fitness Center  0.06
1         Auto Workshop  0.06
2           Pizza Place  0.06
3                  Park  0.06
4      Reco

9     Sushi Restaurant  0.06


-------The Annex, North Midtown, Yorkville----
               venue  freq
0               Café  0.15
1     Sandwich Place  0.15
2        Coffee Shop  0.10
3           Pharmacy  0.05
4        Pizza Place  0.05
5       Burger Joint  0.05
6       Liquor Store  0.05
7         Donut Shop  0.05
8  Indian Restaurant  0.05
9          BBQ Joint  0.05


-------The Beaches----
                      venue  freq
0         Health Food Store  0.25
1                       Pub  0.25
2                     Trail  0.25
3   New American Restaurant  0.00
4             Movie Theater  0.00
5                    Market  0.00
6         Martial Arts Dojo  0.00
7  Mediterranean Restaurant  0.00
8               Men's Store  0.00
9        Mexican Restaurant  0.00


-------The Danforth West, Riverdale----
                    venue  freq
0        Greek Restaurant  0.19
1             Coffee Shop  0.07
2      Italian Restaurant  0.07
3              Restaurant  0.05
4  Furniture / Home Stor

In [75]:
dict1={}

for  hood in torento_onehot_grouped["Neighborhood"]:
    val=[]
    #print(f"-------{hood}----")
    temp=torento_onehot_grouped[torento_onehot_grouped["Neighborhood"]==hood].T.reset_index()
    temp.columns=["venue","freq"]
    temp=temp[1:]
    temp["freq"]=round(temp["freq"].astype(float),2)
    val=list(temp.sort_values(by="freq",axis=0,ascending=False).reset_index(drop=True).head(10)["venue"])
    dict1[hood]=val

In [76]:
cols=["No."+str(x)+"_common_Place" for x in range(1,11)]

In [77]:
neighborhoods_venues_sorted=pd.DataFrame(dict1).T

In [78]:
neighborhoods_venues_sorted.columns=cols

In [79]:
neighborhoods_venues_sorted.insert(0,"Neighborhood",list(neighborhoods_venues_sorted.index))

In [80]:
neighborhoods_venues_sorted.reset_index(drop=True,inplace=True)

In [81]:
neighborhoods_venues_sorted.head()

,Neighborhood,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Seafood Restaurant,Beer Bar,Restaurant,Bakery,Farmers Market,Greek Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Furniture / Home Store,Bar,Stadium,Climbing Gym,Restaurant,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Gym / Fitness Center,Auto Workshop,Pizza Place,Park,Recording Studio,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Airport,Rental Car Location,Coffee Shop,Sculpture Garden,Plane,Boat or Ferry
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Bar,Salad Place,Thai Restaurant


In [82]:
torento_onehot_grouped.head()

,Neighborhood,New American Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.014706,0.0,0.0,0.014706,0.0,0.014706


In [83]:
#Training
from  sklearn.cluster import KMeans

#set no of clusters
n_cluster=5
#set gtraining Data
training_Data=torento_onehot_grouped.drop("Neighborhood",axis=1)
#Training the model
cluster_kmean=KMeans(n_clusters=n_cluster,random_state=0).fit(training_Data)
cluster_kmean

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [85]:
#check the labels
cluster_kmean.labels_

array([2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 4, 2, 2, 0, 0, 0, 1, 0, 4, 2,
       2, 2, 2, 2, 4, 3, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0])

In [86]:
#adding cluster into venues tables
neighborhoods_venues_sorted.insert(0,"cluster_lablel",cluster_kmean.labels_)

In [87]:
neighborhoods_venues_sorted

,cluster_lablel,Neighborhood,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
0,2,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Seafood Restaurant,Beer Bar,Restaurant,Bakery,Farmers Market,Greek Restaurant
1,2,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Furniture / Home Store,Bar,Stadium,Climbing Gym,Restaurant,Italian Restaurant
2,0,"Business reply mail Processing Centre, South C...",Gym / Fitness Center,Auto Workshop,Pizza Place,Park,Recording Studio,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Airport,Rental Car Location,Coffee Shop,Sculpture Garden,Plane,Boat or Ferry
4,2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Bar,Salad Place,Thai Restaurant
5,0,Christie,Grocery Store,Café,Park,Italian Restaurant,Candy Store,Nightclub,Baby Store,Athletics & Sports,Restaurant,Diner
6,2,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Pub,Men's Store,Yoga Studio,Hotel
7,2,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Deli / Bodega
8,2,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Gym,Coffee Shop,Italian Restaurant,Café,Farmers Market,Japanese Restaurant
9,2,Davisville North,Gym / Fitness Center,Hotel,Breakfast Spot,Food & Drink Shop,Sandwich Place,Department Store,Park,Organic Grocery,Monument / Landmark,Mediterranean Restaurant


In [88]:
torento_merged=toronto_df2.copy()

In [89]:
torento_merged=pd.merge(torento_merged,neighborhoods_venues_sorted,on="Neighborhood")

In [91]:
torento_merged.set_index("Postal Code",drop=True,inplace=True)

In [92]:
torento_merged

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
Postal Code,,,,,,,,,,,,,,,
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Cosmetics Shop,Spa,Historic Site
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Diner,Yoga Studio,Creperie,Beer Bar,Smoothie Shop,Burrito Place,Café,Sandwich Place,College Auditorium
M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Café,Hotel,Cosmetics Shop,Italian Restaurant,Bubble Tea Shop,Japanese Restaurant,Lingerie Store,Electronics Store
M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Clothing Store,Cocktail Bar,American Restaurant,Cosmetics Shop,Italian Restaurant,Moroccan Restaurant,Breakfast Spot
M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,New American Restaurant,Movie Theater,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant
M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Seafood Restaurant,Beer Bar,Restaurant,Bakery,Farmers Market,Greek Restaurant
M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Bar,Salad Place,Thai Restaurant
M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Italian Restaurant,Candy Store,Nightclub,Baby Store,Athletics & Sports,Restaurant,Diner
M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,2,Coffee Shop,Café,Hotel,Restaurant,Gym,Deli / Bodega,Thai Restaurant,Cosmetics Shop,Bookstore,Steakhouse


In [93]:
torento_merged.columns

Index(['Borough', 'Neighborhood', 'Latitude', 'Longitude', 'cluster_lablel',
       'No.1_common_Place', 'No.2_common_Place', 'No.3_common_Place',
       'No.4_common_Place', 'No.5_common_Place', 'No.6_common_Place',
       'No.7_common_Place', 'No.8_common_Place', 'No.9_common_Place',
       'No.10_common_Place'],
      dtype='object')

In [94]:
# create map of Torronto using latitude and longitude values
map_toronto = folium.Map(location=[torento_merged["Latitude"][0], torento_merged["Longitude"][0]], zoom_start=10)

# set color scheme for the clusters
x = np.arange(n_cluster)
ys = [i + x + (i*x)**2 for i in range(n_cluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng,neighborhood,cluster_label in zip(torento_merged['Latitude'], torento_merged['Longitude'], torento_merged['Neighborhood'],torento_merged["cluster_lablel"]):
    
    label = folium.Popup(str(neighborhood)+"cluster\n"+str(cluster_label), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster_label],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto

# Cluster1

In [97]:
torento_merged[torento_merged["cluster_lablel"]== 0]

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
Postal Code,,,,,,,,,,,,,,,
M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,New American Restaurant,Movie Theater,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant
M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Italian Restaurant,Candy Store,Nightclub,Baby Store,Athletics & Sports,Restaurant,Diner
M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Pharmacy,Bakery,Bank,Music Venue,Grocery Store,Pet Store,Brewery,Park,Bar,Middle Eastern Restaurant
M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,0,Bar,Café,Vietnamese Restaurant,Coffee Shop,Restaurant,Asian Restaurant,Men's Store,Dog Run,Gift Shop,French Restaurant
M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Sandwich Place,Fast Food Restaurant,Italian Restaurant,Pizza Place,Gym,Pub,Restaurant,Movie Theater,Burrito Place
M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,0,Café,Mexican Restaurant,Thai Restaurant,Bakery,Arts & Crafts Store,Fried Chicken Joint,Music Venue,Bar,Gastropub,Discount Store
M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Café,Sandwich Place,Coffee Shop,Pharmacy,Pizza Place,Burger Joint,Liquor Store,Donut Shop,Indian Restaurant,BBQ Joint
M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Café,Restaurant,Bookstore,Sandwich Place,Bar,Bakery,Japanese Restaurant,Flower Shop,French Restaurant,Chinese Restaurant
M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,0,Café,Coffee Shop,Sushi Restaurant,Italian Restaurant,Pub,Pizza Place,Dessert Shop,Bookstore,Spa,Smoothie Shop


# Cluster 2

In [100]:
torento_merged[torento_merged["cluster_lablel"]== 1]

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
Postal Code,,,,,,,,,,,,,,,
M4N,Central Toronto,Lawrence Park,43.72802,-79.38879,1,Park,Swim School,Bus Line,New American Restaurant,Movie Theater,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant


# Cluster 3

In [101]:
torento_merged[torento_merged["cluster_lablel"]== 2]

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
Postal Code,,,,,,,,,,,,,,,
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Cosmetics Shop,Spa,Historic Site
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Diner,Yoga Studio,Creperie,Beer Bar,Smoothie Shop,Burrito Place,Café,Sandwich Place,College Auditorium
M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Café,Hotel,Cosmetics Shop,Italian Restaurant,Bubble Tea Shop,Japanese Restaurant,Lingerie Store,Electronics Store
M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Clothing Store,Cocktail Bar,American Restaurant,Cosmetics Shop,Italian Restaurant,Moroccan Restaurant,Breakfast Spot
M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Seafood Restaurant,Beer Bar,Restaurant,Bakery,Farmers Market,Greek Restaurant
M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Bar,Salad Place,Thai Restaurant
M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,2,Coffee Shop,Café,Hotel,Restaurant,Gym,Deli / Bodega,Thai Restaurant,Cosmetics Shop,Bookstore,Steakhouse
M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,2,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Scenic Lookout,Sporting Goods Shop,Restaurant,Brewery,Music Venue
M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Ice Cream Shop,Grocery Store,Bubble Tea Shop,Café,Caribbean Restaurant


# Cluster 4

In [102]:
torento_merged[torento_merged["cluster_lablel"]== 3]

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
Postal Code,,,,,,,,,,,,,,,
M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Music Venue,Garden,Museum,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


# Cluster 5

In [103]:
torento_merged[torento_merged["cluster_lablel"]== 4]

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
Postal Code,,,,,,,,,,,,,,,
M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,4,Park,Trail,Jewelry Store,Sushi Restaurant,New American Restaurant,Moroccan Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store
M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,4,Park,Trail,New American Restaurant,Moroccan Restaurant,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant
M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,4,Park,Playground,Trail,New American Restaurant,Movie Theater,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant


Observations: Most of the neighborhoods fall into Cluster 4 which are mostly business areas with cafe, restaurants, supermarkets etc. Cluster 2& 3 is just a garden, Cluster 3 are playground and park, Cluster 5 home service and garden and swim school, and lastly Cluster 1 is pub and noraml food service and trail.